In [1]:
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

In [2]:
random.seed = 10

#### Read files

###### Power grid info

In [3]:
# Create buses
num_buses = 118
buses = np.arange(1, num_buses+1)

# Read gen buses
file_path = '../../data/IEEE_Case118/zones/gen_bus.csv'
gen_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read thermal buses
file_path = '../../data/IEEE_Case118/zones/thermal_bus.csv'
thermal_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read wind gen buses
file_path = '../../data/IEEE_Case118/zones/wind_bus.csv'
wind_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read load buses
file_path = '../../data/IEEE_Case118/zones/load_bus.csv'
load_buses = pd.read_csv(file_path, header=None, index_col=None)

#### Create node features

###### Create Boolean idx for thermals, winds, and loads

In [4]:
# Boolean index of thermal buses
thermal_bidx = np.isin(buses, thermal_buses)

# Boolean index of wind buses
wind_bidx = np.isin(buses, wind_buses)

# Boolean index of load buses
load_bidx = np.isin(buses, load_buses)

###### Create node features matrix with correct dimension

In [5]:
# Get thermal gen features (To be determined)
Pmax = pd.read_csv('../../data/IEEE_Case118/gen_params/Pmax.csv',
                   header=None,
                   index_col=None)
Pmin = pd.read_csv('../../data/IEEE_Case118/gen_params/Pmin.csv',
                   header=None,
                   index_col=None)
ramp_30 = pd.read_csv('../../data/IEEE_Case118/gen_params/ramp_30.csv',
                      header=None,
                      index_col=None)
startup_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/startup_cost.csv',
                            header=None,
                            index_col=None)
shutdown_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/shutdown_cost.csv',
                            header=None,
                            index_col=None)
gencost_params = pd.read_csv('../../data/IEEE_Case118/gen_params/gencost_params.csv',
                                header=None,
                                index_col=None)
reserve_qty = pd.read_csv('../../data/IEEE_Case118/gen_params/reserve_qty.csv',
                            header=None,
                            index_col=None)
reserve_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/reserve_cost.csv',
                            header=None,
                            index_col=None)

# Get the Boolean idx of thermals in gens
bidx = np.isin(gen_buses, thermal_buses).reshape((-1))

temp = np.concatenate([Pmax, Pmin, ramp_30, startup_cost, 
                  shutdown_cost, gencost_params], axis=1)

# Create thermal gens features
thermal_gen_features = np.concatenate([temp[bidx], reserve_qty, reserve_cost], axis=1)

###### Assign thermal gens features

In [6]:
# Time steps
nt = 12

# Read wind and load inputs into MATPOWER
num_samples = 10000

# Empty list to store data
x = []

# Get dimension of thermal, wind and load
num_thermal_features = thermal_gen_features.shape[1]
num_wind_features = nt
num_load_features = nt
num_UC = nt

# Empty node feature matrix
node_features = np.zeros((buses.shape[0], 
                          num_thermal_features+
                          num_wind_features+
                          num_load_features+
                          num_UC))

temp1 = num_thermal_features + num_wind_features
temp2 = temp1 + num_load_features

# Assign thermal features
node_features[thermal_bidx, :num_thermal_features] = thermal_gen_features

# Assign wind and load features
for i in range(num_samples):
    # File path
    wind_path = f'../../data/IEEE_Case118/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case118/inputs/load/sample_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None).to_numpy()
        load = pd.read_csv(load_path, header=None, index_col=None).to_numpy()
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, num_thermal_features:temp1] = wind
    node_features[load_bidx, temp1:temp2] = load

    x.append(node_features.copy())

# Assign UC predictions
for i in range(len(x)):
    # Add UC prediction
    UC_path = f'../../src/IEEE_Case118/model_evaluation/UC_pred_all/pred_{i+1}.csv'
    UC = pd.read_csv(UC_path, header=None, index_col=None).to_numpy()
    x[i][:, -num_UC:] = UC.copy()

The file sample_4363 is not found
The file sample_7077 is not found


#### Create PG ground true for training

In [7]:
# Read MATPOWER UC solution
y = []

# Add zeros to make the dimension as [num_buses, nt*2]
all_PG = np.zeros((num_buses, nt*2))

for i in range(num_samples):
    thermal_PG_path = f'../../data/IEEE_Case118/outputs/PG/sample_{i+1}.csv'
    wind_PG_path = f'../../data/IEEE_Case118/outputs/deployed_wind/sample_{i+1}.csv'
    load_PG_path = f'../../data/IEEE_Case118/outputs/deployed_load/sample_{i+1}.csv'
    try:
        thermal_PG = pd.read_csv(thermal_PG_path, header=None, index_col=None).to_numpy()
        wind_PG = pd.read_csv(wind_PG_path, header=None, index_col=None).to_numpy()
        load_PG = pd.read_csv(load_PG_path, header=None, index_col=None).to_numpy()
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    all_PG[thermal_bidx, :nt] = thermal_PG
    all_PG[wind_bidx, :nt] = wind_PG
    all_PG[load_bidx, -nt:] = load_PG

    y.append(all_PG.copy())

The file sample_4363 is not found
The file sample_7077 is not found


#### Create edge index and attributes

In [8]:
# Read branch info
file_path = '../../data/IEEE_Case118/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Get branch index and attr
edge_index = branch[:, :2] - 1
edge_attr = branch[:, 2:]

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)
edge_attr = torch.from_numpy(edge_attr).float()

#### Create train, test and validation dataset

In [9]:
data_list = []

for i in range(len(x)):
    X = torch.from_numpy(x[i]).float()
    Y = torch.from_numpy(y[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

In [10]:
from MyDataset import MyDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case118/PG2'
MyDataset(root=root, data_list=data_list)

Processing...
Done!


MyDataset(9998)